In [1]:
#hide warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.core.display import display
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import FileUpload, Output
import pandas as pd
from keras.models import load_model
from keras.preprocessing import image
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def get_labels():
    #return df of [label, hebrew letter, name] for each letter in order of model
    labels = pd.read_csv("alef_bet_key.csv", header=None)
    labels[0] = labels[0].astype(str)
    labels = labels.sort_values(0)
    labels = labels.reset_index(drop=True)
    return labels

labels = get_labels()

In [4]:
#get the saved model
model = load_model('saved_model')

2021-12-29 13:22:13.717232: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#input uploaded image and get prediction
def predict_from_image(model, img, labels):
    #convert and resize image to match model specifications
    img = Image.open(io.BytesIO(img))
    img = img.resize((64, 64))
    img_batch = np.expand_dims(img, axis=0)
    
    #predict
    pred = model.predict(img_batch)
    
    #get the label
    # https://androidkt.com/get-class-labels-from-predict-method-in-keras/
    #highest probability = predicted label
    prob = np.max(pred)
    label = labels[1][np.argmax(pred)]
    return pred, label, prob

In [6]:
#sort letters
label_nums = [int(i) for i in labels[0]]

In [7]:
#show percentages of predictions
def show_percs(prediction):
    fig = plt.figure(figsize=(15, 10))
    #sort xs and ys in order of letters 
    #https://stackoverflow.com/questions/6618515/sorting-list-based-on-values-from-another-list
    xs = [x for _,x in sorted(zip(label_nums,labels[1]))]
    ys = [y for _,y in sorted(zip(label_nums,prediction[0]))]
    plt.bar(xs, ys)
    plt.show()

In [8]:
#widget to upload image from computer
uploader = FileUpload(accept='.png')
output2 = Output()

In [9]:
#when a new file is uploaded, change output to display the new file
#https://stackoverflow.com/questions/59259414/can-i-define-an-action-on-file-upload-when-using-ipywidgets-fileupload-widget
def on_upload_change(change):
    img_name = list(uploader.value)[0]
    upload = uploader.value
    
    with output2:
        clear_output()
        print(img_name)
        img_content = uploader.value[img_name]['content']
        img = widgets.Image(value=img_content, height=100, width=100)
        display(img)
        
        #predict and display label
        pred, label, prob = predict_from_image(model, img_content, labels)
        print("letter:", label)
        #print("prediction:", pred)
        #if not certain
        if prob < 10:
            print("Anomalous letter")
        
        show_percs(pred)
        
uploader.observe(on_upload_change, names='_counter')
display(uploader, output2)

FileUpload(value={}, accept='.png', description='Upload')

Output()